In [ ]:
import os
import sys
import glob
import pickle
import tables

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from comet_ml.api import API, APIExperiment
from comet_ml.query import Tag

if '..' not in sys.path:
    sys.path.append('..')
from deep_utils import *

#### Find the best experiment given a set of tags

In [ ]:
api = API(api_key = os.environ['COMET_API_KEY'])
workspace = 'danielelinaro'
project_name = 'inertia'
area_IDs = [1, 2]
D = 2
DZA = 60
query = Tag('two-area') & \
        Tag('_'.join([f'area{area_id}' for area_id in area_IDs])) & \
        Tag(f'D={D}') & \
        Tag(f'DZA={DZA}') & \
        Tag('1D_pipeline')
experiments = api.query(workspace, project_name, query, archived=False)
experiment_IDs = []
MAPE = []
val_loss = []
tags =  []
for experiment in experiments:
    ID = experiment.id
    experiment_IDs.append(ID)
    sys.stdout.write(f'Downloading data for experiment ID {ID}... ')
    metrics = experiment.get_metrics()
    sys.stdout.write('done.\n')
    val_loss.append(np.array([float(m['metricValue']) for m in metrics if m['metricName'] == 'val_loss']))
    has_MAPE = False
    for m in metrics:
        if m['metricName'] == 'mape_prediction':
            val = m['metricValue']
            try:
                MAPE.append(float(val))
            except:
                MAPE.append(list(map(float, val[1:-1].split(' ')[:2])))
            has_MAPE = True
            break
    tags.append(experiment.get_tags())
    print(f'  val_loss: {val_loss[-1].min():.4f}')
    if has_MAPE:
        print(f'      MAPE: {MAPE[-1]}%')
    else:
        print('      MAPE: [experiment not terminated]')
    print('      Tags: "{}"'.format('" "'.join(tags[-1])))
# idx = np.argmin(MAPE)
idx = np.argmin([loss.min() for loss in val_loss])
experiment_ID = experiment_IDs[idx]
MAPE = MAPE[idx]
val_loss = val_loss[idx]
tags = tags[idx]
print(f'The best experiment is {experiment_ID[:6]} (val_loss = {val_loss.min():.4f}, MAPE = {MAPE}%).')

In [ ]:
experiments_path = '../experiments/neural_network/'
checkpoint_path = experiments_path + experiment_ID + '/checkpoints/'
checkpoint_files = glob.glob(checkpoint_path + '*.h5')
network_parameters = pickle.load(open(experiments_path + experiment_ID \
                                      + '/parameters.pkl', 'rb'))
epochs = [int(os.path.split(file)[-1].split('.')[1].split('-')[0]) for file in checkpoint_files]
best_checkpoint = checkpoint_files[epochs.index(np.argmin(val_loss) + 1)]
model = keras.models.load_model(best_checkpoint, compile=True)
# try:
#     data_dirs = ['../' + data_dir.format(area_id) for area_id in network_parameters['area_IDs'] \
#                  for data_dir in network_parameters['data_dirs']]
# except:
#     data_dirs = ['../' + data_dir.format(gen_id) for gen_id in network_parameters['generator_IDs'] \
#                  for data_dir in network_parameters['data_dirs']]
data_dirs = ['..' + os.path.sep + os.path.sep.join([d for d in data_dir.split(os.path.sep) if '{}' not in d]) \
             for data_dir in network_parameters['data_dirs']]
# we need mean and standard deviation of the training set to normalize the data
x_train_mean = network_parameters['x_train_mean']
x_train_std  = network_parameters['x_train_std']
data_dir = data_dirs[0]
if not os.path.isdir(data_dir):
    raise Exception(f'{data_dir}: no such directory')
print(f'Loaded network from {best_checkpoint}.')
print(f'Data directory is {data_dir}.')

In [ ]:
keras.utils.plot_model(model, show_shapes=False, dpi=96)

### Constant inertia
1. the value of inertia is either 3.5 or 8.5, for both areas
1. **omega** and **Pe** of both areas used for the estimation

In [ ]:
default_H = [6.5, 6.175]
bus_IDs = [7, 9]

window_dur = 60
window_step = 10

H_values = [3.5, 8.5]
N_H = len(H_values)

data_normalized = [[] for _ in range(len(bus_IDs))]
inertia = [[] for _ in range(len(bus_IDs))]

var_names = [var_name.format(bus_ID) for bus_ID in bus_IDs for var_name in ['omegael_bus{}', 'Pe_bus{}']]
data_mean = {var_name: x_train_mean[k] for k,var_name in enumerate(var_names)}
data_std = {var_name: x_train_std[k] for k,var_name in enumerate(var_names)}

H = [0,0]
for i, bus_to_predict in enumerate(bus_IDs):
    other_bus = bus_IDs[1 - i]
    for j,h in enumerate(H_values):
        H[i] = h
        H[1-i] = default_H[1-i]
        data_file = data_dir + f'/inertia_{H[0]:.3f}_{H[0]:.3f}_{H[1]:.3f}_{H[1]:.3f}.h5'

        t, _, data_norm, data_sliding, _ = load_data_slide([data_file],
                                                            var_names,
                                                            data_mean,
                                                            data_std,
                                                            window_dur,
                                                            window_step,
                                                            verbose = True)
        data_normalized[i].append(data_norm)
        dt = np.diff(t[:2])[0]
        time, HH, _ = predict(model, data_sliding, window_step, dt)
        inertia[i].append(HH)

In [ ]:
fig = plt.figure(figsize=(10, (N_H + 2) * 3))
gs = fig.add_gridspec(N_H * 2 + 2, 4)
ax = []
for i in range(0, len(H_values)*2, 2):
    ax.append([fig.add_subplot(gs[i, :3]), fig.add_subplot(gs[i, 3])]),
    ax.append([fig.add_subplot(gs[i+1, :3]), fig.add_subplot(gs[i+1, 3])]),
ax.append([fig.add_subplot(gs[-2:,:2]), fig.add_subplot(gs[-2:,2:])])

col = [[.2,.2,.2], [.8,0,0]]

idx = t < 60
var_names = 'omegael_bus{}','Pe_bus{}'
for i,bus_ID in enumerate(bus_IDs):
    for j,h in enumerate(H_values):
        ax[-1][i].plot(time[[0,-1]] / 60, h + np.zeros(2), '--', color=col[i], lw=1)
        ax[-1][i].plot(time[[0,-1]] / 60, default_H[1-i] + np.zeros(2), '--', color=col[1-i], lw=1)
        for k,var_name in enumerate(var_names):
            key = var_name.format(bus_ID)
            value = data_normalized[i][j][key]
            n,edges = np.histogram(value, bins=31, range=(-3,3), density=True)
            m = j * len(var_names) + k
            ax[m][0].plot(t[idx], value[idx], color=col[i], lw=1, label=key.split('_')[1])
            ax[m][1].plot(n, edges[:-1] + np.diff(edges[:2])[0] / 2, color=col[i], lw=1)
            ax[-1][i].plot(time / 60, inertia[i][j][:,k], color=col[k], lw=1)
        ax[j * 2][0].set_title(f'H = {h} /s')
    ax[-1][i].set_xlabel('Time [min]')
    
for a in ax:
    for side in 'right','top':
        for i in range(2):
            a[i].spines[side].set_visible(False)

ax[0][0].legend(loc='best')
for i in range(len(ax)):
    if i < len(ax) - 2:
        ax[i][0].get_shared_x_axes().join(ax[i][0], ax[i+1][0])
    ax[i][0].get_shared_y_axes().join(ax[i][0], ax[i][1])
    ax[i][1].set_yticklabels([])

for i in range(0, len(ax)-1, 2):
    ax[i][0].set_ylabel(r'$\omega_{e}$')
    ax[i+1][0].set_ylabel(r'$P_{e}$')
ax[-1][0].get_shared_x_axes().join(ax[-1][0], ax[-1][1])

ax[-1][0].set_xlim([0,30])
ax[-1][0].set_ylim([2.5,10])
ax[-1][0].set_ylabel(r'Inertia [$s^{-1}$]')
ax[-2][0].set_xlabel('Time [s]')
ax[-2][1].set_xlabel('Fraction')
fig.tight_layout()
fig.savefig('multi_area_prediction_const_H.pdf')

### Step of inertia
1. H steps **instantaneously** from 3.5 to 3.8
1. **omega** and **Pe** of the both areas used for the estimation
1. two simulations joined together (i.e., **no transient** during the step)

In [ ]:
default_H = [6.5, 6.175]
bus_IDs = [7, 9]

window_dur = 60
window_step = 10

H_values = [3.5, 3.8]
N_H = len(H_values)

data_normalized = []
inertia = []

H = [0, 0]

var_names = [var_name.format(bus_ID) for bus_ID in bus_IDs for var_name in ['omegael_bus{}', 'Pe_bus{}']]
data_mean = {var_name: x_train_mean[k] for k,var_name in enumerate(var_names)}
data_std = {var_name: x_train_std[k] for k,var_name in enumerate(var_names)}

for i, bus_to_predict in enumerate(bus_IDs):
    other_bus = bus_IDs[1 - i]
    data_files = []
    for j,h in enumerate(H_values):
        H[i] = h
        H[1-i] = default_H[1-i]
        data_files.append(data_dir + f'/inertia_{H[0]:.3f}_{H[0]:.3f}_{H[1]:.3f}_{H[1]:.3f}.h5')

    t, _, data_norm, data_sliding, _ = load_data_slide(data_files,
                                                       var_names,
                                                       data_mean,
                                                       data_std,
                                                       window_dur,
                                                       window_step,
                                                       verbose = True)
    data_normalized.append(data_norm)
    dt = np.diff(t[:2])[0]
    time, HH, _ = predict(model, data_sliding, window_step, dt)
    inertia.append(HH)

In [ ]:
t0 = 3600
window = 120
idx = (t > t0 - window) & (t < t0 + window)
fig = plt.figure(figsize = (10,8))
gs = fig.add_gridspec(4, 2)
ax = [
    [fig.add_subplot(gs[0,0]), fig.add_subplot(gs[0,1])],
    [fig.add_subplot(gs[1,0]), fig.add_subplot(gs[1,1])],
    [fig.add_subplot(gs[2:,0]), fig.add_subplot(gs[2:,1])],
]

ax[0][0].plot(t[idx], data_normalized[0]['omegael_bus7'][idx], color=col[0], lw=1, label='bus7')
ax[0][0].plot(t[idx], data_normalized[0]['omegael_bus9'][idx], color=col[1], lw=1, label='bus9')
ax[1][0].plot(t[idx], data_normalized[0]['Pe_bus9'][idx], color=col[1], lw=1)
ax[1][0].plot(t[idx], data_normalized[0]['Pe_bus7'][idx], color=col[0], lw=1)

ax[2][0].plot(time[[0,time.size//2]] / 60, H_values[0] + np.zeros(2), '--', color=col[0], lw=1)
ax[2][0].plot(time[[time.size//2,-1]] / 60, H_values[1] + np.zeros(2), '--', color=col[0], lw=1)
ax[2][0].plot(time / 60, inertia[0][:,0], color=col[0])

ax[2][0].plot(time[[0,-1]] / 60, default_H[1] + np.zeros(2), '--', color=col[1], lw=1)
ax[2][0].plot(time / 60, inertia[0][:,1], color=col[1])

ax[0][1].plot(t[idx], data_normalized[1]['omegael_bus7'][idx], color=col[0], lw=1)
ax[0][1].plot(t[idx], data_normalized[1]['omegael_bus9'][idx], color=col[1], lw=1)
ax[1][1].plot(t[idx], data_normalized[1]['Pe_bus7'][idx], color=col[0], lw=1)
ax[1][1].plot(t[idx], data_normalized[1]['Pe_bus9'][idx], color=col[1], lw=1)

ax[2][1].plot(time[[0,time.size//2]] / 60, H_values[0] + np.zeros(2), '--', color=col[1], lw=1)
ax[2][1].plot(time[[time.size//2,-1]] / 60, H_values[1] + np.zeros(2), '--', color=col[1], lw=1)
ax[2][1].plot(time / 60, inertia[1][:,1], color=col[1])

ax[2][1].plot(time[[0,-1]] / 60, default_H[0] + np.zeros(2), '--', color=col[0], lw=1)
ax[2][1].plot(time / 60, inertia[1][:,0], color=col[0])

ax[2][0].set_xlabel('Time [min]')
ax[2][1].set_xlabel('Time [min]')

ax[0][0].legend(loc='lower right')

limits = [[-4,4], [-4,4], [3,7]]
for i in range(3):
    ax[i][0].get_shared_x_axes().join(ax[i][0], ax[i][1])
    ax[i][0].get_shared_y_axes().join(ax[i][0], ax[i][1])
    for a in ax[i]:
        if i == 2:
            a.plot(t0 / 60 + np.zeros(2), limits[i], '--', lw=1, color=[.6,.6,.6])
        else:
            a.plot(t0 + np.zeros(2), limits[i], '--', lw=1, color=[.6,.6,.6])
        for side in 'right', 'top':
            a.spines[side].set_visible(False)
    if i < 2:
        ax[0][i].set_xticklabels([])
    ax[i][1].set_yticklabels([])

ax[0][0].set_ylabel(r'$\omega_{e}$')
ax[1][0].set_ylabel(r'$P_{e}$')
ax[2][0].set_ylabel(r'Inertia [$s^{-1}$]')

fig.tight_layout()

fig.savefig('multi_area_prediction_step_H.pdf')

### Steps of inertia with transient
1. the inertia of area 1 steps from 6.5 to 5.5 at 20 s
1. the inertia of area 2 steps from 6.175 to 6.5 at 40 s
1. **omega** and **Pe** of the both areas used for the estimation
1. one single simulation

In [ ]:
data_file = data_dir + '/inertia_steps.h5'
bus_IDs = [7, 9]

window_dur = 60
window_step = 10

var_names = [var_name.format(bus_ID) for bus_ID in bus_IDs for var_name in ['omegael_bus{}', 'Pe_bus{}']]
data_mean = {var_name: x_train_mean[k] for k,var_name in enumerate(var_names)}
data_std = {var_name: x_train_std[k] for k,var_name in enumerate(var_names)}

t, _, data_normalized, data_sliding, _ = load_data_slide([data_file],
                                                         var_names,
                                                         data_mean,
                                                         data_std,
                                                         window_dur,
                                                         window_step,
                                                         verbose = True)

dt = np.diff(t[:2])[0]
time, inertia, _ = predict(model, data_sliding, window_step, dt)

In [ ]:
fig = plt.figure(figsize = (6,9))
gs = fig.add_gridspec(8, 1)
ax = [
    fig.add_subplot(gs[:2,0]),
    fig.add_subplot(gs[2:4,0]),
    fig.add_subplot(gs[4:-1,0]),
    fig.add_subplot(gs[-1,0])
]

ds = 5
ax[0].plot(t[::ds] / 60, data_normalized['omegael_bus7'][::ds], color=col[0], lw=1, label='bus7')
ax[0].plot(t[::ds] / 60, data_normalized['omegael_bus9'][::ds], color=col[1], lw=1, label='bus9')
ax[1].plot(t[::ds] / 60, data_normalized['Pe_bus7'][::ds], color=col[0], lw=1)
ax[1].plot(t[::ds] / 60, data_normalized['Pe_bus9'][::ds], color=col[1], lw=1)
ax[0].set_ylim([-5,5])
ax[1].set_ylim([-5,5])

ax[2].plot(time[::ds] / 60, inertia[::ds,0], color=col[0])
ax[2].plot(time[::ds] / 60, inertia[::ds,1], color=col[1])
ax[2].set_ylim([4.5,7.5])
ax[2].plot([0, 20], 6.5 + np.zeros(2), '--', color=col[0], lw=1)
ax[2].plot([20, 60], 5.5 + np.zeros(2), '--', color=col[0], lw=1)
ax[2].plot([0, 40], 6.175 + np.zeros(2), '--', color=col[1], lw=1)
ax[2].plot([40, 60], 6.5 + np.zeros(2), '--', color=col[1], lw=1)

ax[3].plot([0, 20], 6.5 + np.zeros(2), '-', color=col[0], lw=2)
ax[3].plot([20, 60], 5.5 + np.zeros(2), '-', color=col[0], lw=2)
ax[3].plot([0, 40], 6.175 + np.zeros(2), '-', color=col[1], lw=2)
ax[3].plot([40, 60], 6.5 + np.zeros(2), '-', color=col[1], lw=2)
ax[3].plot(20 + np.zeros(2), [4.5, 7.5], '--', color=col[0], lw=1)
ax[3].plot(40 + np.zeros(2), [4.5, 7.5], '--', color=col[1], lw=1)
ax[3].set_ylim([4.5,7.5])
ax[3].set_xlabel('Time [min]')

ax[0].legend(loc='lower right')

for a in ax:
    a.set_xlim([-1,61])
    for side in 'right', 'top':
        a.spines[side].set_visible(False)
    if a != ax[-1]:
        a.set_xticklabels([])

ax[0].set_ylabel(r'$\omega_{e}$')
ax[1].set_ylabel(r'$P_{e}$')
ax[2].set_ylabel(r'Inertia [$s^{-1}$]')
ax[3].set_ylabel(r'Inertia [$s^{-1}$]')

fig.tight_layout()
fig.savefig('multi_area_prediction_steps_H.pdf')

### Three generators in area 1

In [ ]:
H = [6.5, 6.175, 9.75]
P_nom = [0.5, 1.0]
bus_ID = 6

data_files = [
    data_dir + f'/inertia_{H[0]:.3f}_{H[0]:.3f}_{H[1]:.3f}_{H[1]:.3f}.h5',
    data_dir + f'/inertia_{H[2]:.3f}_{H[2]:.3f}_{H[1]:.3f}_{H[1]:.3f}.h5',
    data_dir + f'/inertia_{H[0]:.3f}_{H[0]:.3f}_{H[0]:.3f}_{H[1]:.3f}_{H[1]:.3f}_P2ab={P_nom[0]:.1f}GW_bus{bus_ID}.h5',
    data_dir + f'/inertia_{H[0]:.3f}_{H[0]:.3f}_{H[0]:.3f}_{H[1]:.3f}_{H[1]:.3f}_P2ab={P_nom[1]:.1f}GW_bus{bus_ID}.h5'#,
]
labels = [f'2 gen H={H[0]}/s P={P_nom[1]:.1f}GW',
          f'2 gen H={H[2]}/s P={P_nom[1]:.1f}GW',
          f'3 gen H={H[0]}/s P={P_nom[0]:.1f}GW',
          f'3 gen H={H[0]}/s P={P_nom[1]:.1f}GW'#,
]

bus_IDs = [7, 9]

window_dur = 60
window_step = 10

var_names = [var_name.format(bus_ID) for bus_ID in bus_IDs for var_name in ['omegael_bus{}', 'Pe_bus{}']]
data_mean = {var_name: x_train_mean[k] for k,var_name in enumerate(var_names)}
data_std = {var_name: x_train_std[k] for k,var_name in enumerate(var_names)}

t = []
data_normalized = []
data_sliding = []
time = []
inertia = []

for data_file in data_files:
    tt, _, data_norm, data_slid, _ = load_data_slide([data_file],
                                                      var_names,
                                                      data_mean,
                                                      data_std,
                                                      window_dur,
                                                      window_step,
                                                      verbose = True)
    t.append(tt)
    data_normalized.append(data_norm)
    data_sliding.append(data_slid)
    dt = np.diff(tt[:2])[0]

    tt, hh, _ = predict(model, data_slid, window_step, dt)
    time.append(tt)
    inertia.append(hh)

In [ ]:
col = [[0,0,0], [.8,0,.8], [1,0.5,0], [0,.8,0], [0,0,1]]

fig = plt.figure(figsize=(8,4))
gs = fig.add_gridspec(2, 2)
ax = [
    fig.add_subplot(gs[0,0]),
    fig.add_subplot(gs[1,0]),
    fig.add_subplot(gs[:,1])
]

for i,(tt,hh) in enumerate(zip(time, inertia)):
    for j in range(2):
        if j == 1:
            ax[2].plot(tt / 60, hh[:,j], color=col[i], lw=1, label=labels[i])
        else:
            ax[1 - i % 2].plot(tt / 60, hh[:,j], color=col[i], lw=1, label=labels[i])

ax[0].plot(time[0][[0,-1]] / 60, H[2] + np.zeros(2), '--', color=[1,1,.2], lw=2)
ax[1].plot(time[0][[0,-1]] / 60, H[0] + np.zeros(2), '--', color=[1,1,.2], lw=2)
ax[2].plot(time[0][[0,-1]] / 60, H[1] + np.zeros(2), '--', color=[1,1,.2], lw=2)


ax[0].legend(loc='best', fontsize=6)
ax[1].legend(loc='best', fontsize=6)
ax[0].set_title('Area 1')
ax[2].set_title('Area 2')
for a in ax:
    for side in 'top','right':
        a.spines[side].set_visible(False)
    a.set_ylabel('Inertia [1/s]')
ax[1].set_xlabel('Time [min]')
ax[2].set_xlabel('Time [min]')

for i in 1,2:
    ax[0].get_shared_x_axes().join(ax[0], ax[i])

ax[0].set_xlim([0, 60])
ax[0].set_xticklabels([])

fig.tight_layout()
fig.savefig(f'two-area_three_gen_bus_{bus_ID}_inertia_estimation.pdf')

In [ ]:
var_names = ['omegael_bus7', 'Pe_bus7']
fig,ax = plt.subplots(1, 2, sharex=True, figsize=(8,3))
j = 0
for i,data_norm in enumerate(data_normalized):
    for j,var_name in enumerate(var_names):
        y, edges = np.histogram(data_norm[var_name], bins=31, range=(-3,3), density=True)
        x = edges[:-1] + np.diff(edges)[0] / 2
        ax[j].plot(x, y, color=col[i], lw=1, label=labels[i])
ax[0].legend(loc='lower left', fontsize=6)

for a in ax:
    for side in 'top','right':
        a.spines[side].set_visible(False)
ax[0].set_xlabel(r'Norm. $\omega_{e}$')
ax[1].set_xlabel(r'Norm. $P_{e}$')
ax[0].set_ylabel('Fraction')

fig.tight_layout()
fig.savefig(f'two-area_three_gen_bus_{bus_ID}_var_distr.pdf')